In [12]:
current_dir = os.getcwd()

# Specify the relative paths
DATA_PATH = os.path.join(current_dir, "data", "preprocessed_data.pkl")
ROLES_PATH = os.path.join(current_dir, "data", "roles_short_names.csv")

In [11]:
import pandas as pd 
import numpy as np
import pickle
import os

In [13]:
df = pd.read_pickle(DATA_PATH)
roles_names = pd.read_csv(ROLES_PATH, sep=';')

In [15]:
roles_names

,Original name,Short name
0,"Developer, back-end",Back-end dev
1,"Developer, full-stack",Full-stack dev
2,"Developer, front-end",Front-end dev
3,"Developer, desktop or enterprise applications",Desktop dev
4,"Developer, mobile",Mobile dev
5,DevOps specialist,DevOps
6,Database administrator,Database admin
7,Designer,Designer
8,System administrator,System admin
9,"Developer, embedded applications or devices",Embedded dev
